**Pull a Random Sample of Instances from PostgreSQL Database Table**

This file samples 50,000 instances of the 2017 HMDA Data Set (All Records) to inform initial models, using balanced 50/50 classes for the outcome variable.

In [1]:
import os
import pandas as pd
import psycopg2

**Instantiate the connection and cursor objects using the PostgreSQL database of your choice,then verify the database and tables in the database.**

In [2]:
con = psycopg2.connect(host="",database="", user="", password="", port="5432")
cur = con.cursor()

In [3]:
print('Current database connection:')
cur.execute('SELECT current_database();')
current_db = cur.fetchone()
print(current_db)

Current database connection:
('mortgages',)


In [4]:
cur.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('test_table',)
('hmda_lar_2015_allrecords',)
('hmda_lar_2017_allrecords',)
('hmda_lar_2013_allrecords',)
('hmda_lar_2016_allrecords',)


**Read query result into a pandas dataframe to call the decribe() method.**
Use "ORDER BY RANDOM()" in PostgreSQL to pull a random sample of instances.

In [6]:
query_statement = '''SELECT * FROM hmda_lar_2017_allrecords 
                     WHERE action_taken_name IN (
                        'Loan originated',
                        'Loan purchased by the institution'
                        )
                    ORDER BY RANDOM() 
                    LIMIT 25000;'''
df_positive_outcome = pd.read_sql_query(query_statement, con)

In [7]:
df_positive_outcome.describe(include='all')

,tract_to_msamd_income,rate_spread,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,state_name,...,applicant_sex_name,applicant_race_name_5,applicant_race_name_4,applicant_race_name_3,applicant_race_name_2,applicant_race_name_1,applicant_ethnicity_name,agency_name,agency_abbr,action_taken_name
count,25000.000000,25000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,25000,25000,...,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000
unique,NaN,314,NaN,NaN,NaN,NaN,NaN,NaN,700,52,...,4,2,2,3,6,7,4,6,6,2
top,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,California,...,Male,,,,,White,Not Hispanic or Latino,Department of Housing and Urban Development,HUD,Loan originated
freq,NaN,23681,NaN,NaN,NaN,NaN,NaN,NaN,4083,2923,...,13644,24999,24999,24994,24880,16058,16905,11476,11476,19417
mean,113.682619,NaN,5703.061240,31.221668,1468.833560,1993.204640,249.406520,71365.74800,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,40.453419,NaN,3116.392942,25.372575,871.808851,1067.080257,474.900927,14778.33165,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,160.000000,0.000000,0.000000,10.000000,1.000000,18000.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,87.230003,NaN,3861.750000,11.040000,933.000000,1342.000000,124.000000,62000.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,108.269997,NaN,5162.000000,23.150000,1325.000000,1814.000000,195.000000,68500.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,134.149994,NaN,6771.000000,45.369999,1813.000000,2409.000000,300.000000,77500.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pd.crosstab(index=df_positive_outcome['action_taken_name'], columns="count", margins=True)

col_0,count,All
action_taken_name,,
Loan originated,19417,19417
Loan purchased by the institution,5583,5583
All,25000,25000


In [11]:
#NOTE: this excludes 'application approved but not accepted' and 'application withdrawn by applicant'
query_statement = '''SELECT * FROM hmda_lar_2017_allrecords 
                    WHERE action_taken_name IN (
                        'Application denied by financial institution',
                        'File closed for incompleteness',
                        'Preapproval request denied by financial institution'
                        )
                    ORDER BY RANDOM() 
                    LIMIT 25000;'''
df_negative_outcome = pd.read_sql_query(query_statement, con)

In [13]:
df_negative_outcome.describe(include='all')

,tract_to_msamd_income,rate_spread,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,state_name,...,applicant_sex_name,applicant_race_name_5,applicant_race_name_4,applicant_race_name_3,applicant_race_name_2,applicant_race_name_1,applicant_ethnicity_name,agency_name,agency_abbr,action_taken_name
count,25000.000000,25000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000,25000,...,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000
unique,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,617,52,...,4,3,4,5,6,7,4,6,6,3
top,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,California,...,Male,,,,,White,Not Hispanic or Latino,Department of Housing and Urban Development,HUD,Application denied by financial institution
freq,NaN,25000,NaN,NaN,NaN,NaN,NaN,NaN,1671,3007,...,14611,24996,24995,24982,24760,16070,17768,13341,13341,19414
mean,107.837498,NaN,5564.375360,35.844718,1399.873000,1961.388000,207.509560,69639.144000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,40.275996,NaN,2902.659035,28.392458,792.761686,999.409469,687.738525,15260.092875,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,87.000000,0.000000,0.000000,0.000000,1.000000,15800.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,82.330002,NaN,3785.000000,12.120000,886.000000,1322.000000,72.000000,59800.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,102.680000,NaN,5073.500000,27.250000,1279.000000,1803.000000,146.000000,67500.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,126.540001,NaN,6677.000000,55.070000,1751.000000,2388.000000,252.000000,75200.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
pd.crosstab(index=df_positive_outcome['action_taken_name'], columns="count", margins=True)

col_0,count,All
action_taken_name,,
Loan originated,19417,19417
Loan purchased by the institution,5583,5583
All,25000,25000


**Append DataFrames, then write DataFrame to CSV, and close PostgreSQL cursor and connection objects.**

In [16]:
df = df_positive_outcome.append(df_negative_outcome)

In [17]:
df.describe()

,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,census_tract_number,as_of_year
count,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.0
mean,110.760058,5633.718300,33.533193,1434.35328,1977.296320,228.458040,70502.446000,2261.724428,2017.0
std,40.470068,3012.190961,27.023663,833.92800,1033.910772,591.345455,15045.781898,3258.549822,0.0
min,0.000000,87.000000,0.000000,0.00000,0.000000,1.000000,15800.000000,1.000000,2017.0
25%,84.839996,3826.000000,11.530000,909.00000,1332.000000,97.000000,61100.000000,104.020000,2017.0
50%,105.470001,5120.000000,25.160000,1301.00000,1808.000000,172.000000,67900.000000,408.090000,2017.0
75%,130.552502,6726.000000,50.049999,1782.00000,2397.000000,279.000000,76300.000000,3735.000000,2017.0
max,468.000000,53812.000000,100.000000,19529.00000,25391.000000,68000.000000,131500.000000,9856.000000,2017.0


In [18]:
filepath = os.path.abspath(os.path.join( "..", "fixtures", "hmda2017sample_balanced.csv"))
df.to_csv(filepath)
cur.close()
con.close()